In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
base_casas = pd.read_csv('/home/sagemaker-user/regressao/house_prices.csv')
base_casas

In [ ]:
base_casas.isnull().sum()

In [ ]:
base_casas.corr()

In [ ]:
base_casas.dtypes

In [26]:
base_casas['date'] = pd.to_datetime(base_casas['date'], format='%Y%m%dT%H%M%S')

In [ ]:
figura = plt.figure(figsize=(20,20))
sns.heatmap(base_casas.corr(),annot= True);

In [32]:
X = base_casas.iloc[:, 3:19].values

In [34]:
y = base_casas.iloc[:, 2].values
y

array([221900., 538000., 180000., ..., 402101., 400000., 325000.])

In [35]:
X = np.array(X).astype('float32')
y = np.array(y).astype('float32')

In [36]:
from sklearn.model_selection import train_test_split

In [37]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size=0.3, random_state=0)

In [39]:
X_treinamento.shape, X_teste.shape

((15129, 16), (6484, 16))

In [40]:
y_treinamento.shape, y_teste.shape

((15129,), (6484,))

In [41]:
import sagemaker
import boto3
from sagemaker import Session

In [42]:
session = sagemaker.Session
bucket = 'curso-aws-sagemaker1996'
subpasta_modelo = 'modelos/house-prices/linear-learner'
subpasta_dataset = 'datasets/house-prices'

In [43]:
role = sagemaker.get_execution_role()
print(role)

arn:aws:iam::774305611476:role/service-role/AmazonSageMaker-ExecutionRole-20250829T201890


In [44]:
import io
import sagemaker.amazon.common as smac

buffer = io.BytesIO()
smac.write_numpy_to_dense_tensor(buffer, X_treinamento, y_treinamento)
buffer.seek(0)

0

In [45]:
import os
key = 'house-train-data'
boto3.resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key)).upload_fileobj(buffer)

In [48]:
os.path.join(subpasta_dataset, 'train', key)

'datasets/house-prices/train/house-train-data'

In [49]:
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key)
print('Localização da pasta de treinamento:{}'.format(s3_train_data))

Localização da pasta de treinamento:s3://curso-aws-sagemaker1996/datasets/house-prices/train/house-train-data


In [50]:
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)
print('Modelo será salvo em:{}'.format(output_location))

Modelo será salvo em:s3://curso-aws-sagemaker1996/modelos/house-prices/linear-learner/output
